# 🐶🐱 SSD 기반 개/고양이 얼굴(Object Detection) 검출

## 📌 1. 프로젝트 목적
Oxford-IIIT Pet 데이터셋의 바운딩박스(XML) 어노테이션을 이용해,
이미지에서 개/고양이의 얼굴(Face) 영역을 검출하는 Object Detection 모델을 구축하는 것이다.

분류(Classification)가 "이미지 1장 → 라벨 1개"라면,
탐지(Object Detection)는 "이미지 1장 → (바운딩박스 여러 개 + 클래스)"를 예측해야 한다.
따라서 모델은 다음 두 가지를 동시에 잘해야 한다.

- 어디에 있는가(위치): 박스를 정확히 맞추기
- 무엇인가(분류): 박스가 cat인지 dog인지 맞추기


## 2. 데이터
- 데이터셋: The Oxford-IIIT Pet Dataset (Kaggle mirror 사용)
- 구성
  - images: 원본 이미지(.jpg)
  - annotations/xmls: 바운딩박스 어노테이션(.xml)
  - trainval.txt / test.txt: 학습/테스트 분할 정보
- 실제 확인 결과
  - Train/Validation 데이터 수: 3680
  - Test 데이터 수: 3669
  - XML 파일 개수: 3686


## 3. 전체 로드맵
- 모델: SSD300(VGG16 backbone)
  - SSD는 “실시간(빠른)” 탐지 모델 계열로 구현이 표준화되어 있고, torchvision에서 안정적으로 제공된다.

- Full Fine-tuning
  - 얼굴 검출은 일반 객체보다 패턴이 비교적 명확해, 사전학습 가중치에서 전체를 미세조정하면 빠르게 수렴하는 경우가 많다.

- 평가: IoU + AP + mAP@0.5
  - 탐지는 "맞췄다/틀렸다"를 단순 정확도로 표현하기 어렵다. IoU로 “겹침 정도”를 기준 삼고, 그 기준 위에서 AP/mAP로 종합 성능을 평가한다.

- 데이터 불균형 대응
  - WHY: 실제로 cat/dog 개수가 차이가 나서(체감상 2배 수준), 학습이 한 쪽 클래스에 치우칠 가능성이 있다. 그래서 WeightedRandomSampler로 샘플링 단계에서 균형을 보정한다.



▫️라이브러리 설치 & 임포트

In [ ]:
# torchvision detection: SSD300 구현이 안정적으로 제공됨
# transforms.v2 + tv_tensors: bbox(박스)까지 같이 안전하게 증강하려고 사용

!pip -q install kagglehub
import kagglehub

import os
import re
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xml.etree.ElementTree as ET

from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import WeightedRandomSampler

import torchvision
import torchvision.transforms.v2 as v2
import torchvision.tv_tensors as tv_tensors

from torchvision.models.detection import ssd300_vgg16
from torchvision.models.detection.ssd import SSDClassificationHead
from torchvision.models.detection.ssd import SSD300_VGG16_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


▫️데이터 다운로드 & 경로 준비

In [ ]:
path = kagglehub.dataset_download("zippyz/cats-and-dogs-breeds-classification-oxford-dataset")
print("Path to dataset files:", path)

image_dir = os.path.join(path, "images", "images")
xml_dir   = os.path.join(path, "annotations", "annotations", "xmls")

trainval_txt = os.path.join(path, "annotations", "annotations", "trainval.txt")
test_txt     = os.path.join(path, "annotations", "annotations", "test.txt")


▫️데이터 분할 파일(trainval/test) 읽고 정합성 체크

In [ ]:
import pandas as pd

# 탐지에서 흔한 에러: "이미지는 있는데 xml이 없거나", "xml은 있는데 이미지가 없는" 케이스
# 학습 중간에 터지기 전에, 초반에 데이터 정합성을 먼저 검사하는 게 안전하다.

df_trainval = pd.read_csv(trainval_txt, sep=r"\s+", header=None)
df_trainval.columns = ["Image", "ClassID", "Species", "BreedID"]

df_test = pd.read_csv(test_txt, sep=r"\s+", header=None)
df_test.columns = ["Image", "ClassID", "Species", "BreedID"]

print("Train/Validation 데이터 수:", len(df_trainval))
print("Test 데이터 수:", len(df_test))

xml_files = [f for f in os.listdir(xml_dir) if f.endswith(".xml")]
print("XML 파일 개수:", len(xml_files))


trainval_list = df_trainval["Image"].tolist()
test_list     = df_test["Image"].tolist()

# 정합성 체크: 이미지/어노테이션 존재 여부
def exists_pair(stem):
    img_ok = os.path.exists(os.path.join(image_dir, stem + ".jpg"))
    xml_ok = os.path.exists(os.path.join(xml_dir, stem + ".xml"))
    return img_ok, xml_ok

# 샘플 몇 개만 빠르게 확인
for stem in trainval_list[:5]:
    print(stem, exists_pair(stem))


✅ XML 파싱을 먼저 확인하는 이유

- Object Detection은 정답이 "박스 좌표"이다.
- 즉, XML 파서가 틀리면 모델이 아무리 학습돼도 결과가 전부 망가진다.

그래서 아래 과정은 단순한 절차가 아니라,
"학습 품질을 결정하는 핵심 기반"이라고 보는 게 맞다.

1) XML에서 (xmin, ymin, xmax, ymax) 좌표가 제대로 읽히는지  
2) 이미지 위에 박스를 직접 그려서 눈으로 맞는지  
3) 라벨(cat/dog)이 기대대로 들어오는지


▫️XML 파서 + 라벨 규칙 정의

In [ ]:
# SSD는 background를 0으로 두는 관례가 있다. 실제 객체 클래스는 1부터 시작하는게 안전하다.
# torchvision detection 계열 대부분이 이 방식을 전제로 한다.

classes = ["background", "cat", "dog"]  # label id: 0,1,2
NUM_CLASSES = len(classes)

def parse_xml(xml_path):
    """
    XML 파일에서 바운딩박스와 라벨을 추출한다.

    반환 형태(관례):
    - boxes: (N,4) 텐서, 좌표는 [xmin, ymin, xmax, ymax] (XYXY)
    - labels: (N,) 텐서, label id는 1(cat) 또는 2(dog)
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    boxes = []
    labels = []

    # Oxford Pet XML 구조는 object 태그에 bbox가 들어있다.
    # species 등 메타를 쓰기도 하지만, 여기서는 "cat vs dog"만 필요.
    for obj in root.findall("object"):
        name = obj.find("name").text.lower()  # 예: "cat" 또는 "dog" 형태로 들어오는 경우가 많음

        bnd = obj.find("bndbox")
        xmin = float(bnd.find("xmin").text)
        ymin = float(bnd.find("ymin").text)
        xmax = float(bnd.find("xmax").text)
        ymax = float(bnd.find("ymax").text)

        boxes.append([xmin, ymin, xmax, ymax])

        # 라벨 매핑 (데이터에 따라 name 표기 형태가 다르면 여기만 수정)
        if "cat" in name:
            labels.append(1)
        else:
            labels.append(2)

    boxes = torch.tensor(boxes, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.int64)
    return boxes, labels


▫️샘플 1장에 대해 “정답 박스” 시각화로 sanity check

In [ ]:
# 탐지는 수치보다 "처음엔" 눈으로 확인하는 게 훨씬 빠르다.
# 박스가 어긋나면 학습을 epoch을 많이 돌려도 의미가 없다.

def show_image_with_boxes(stem):
    img_path = os.path.join(image_dir, stem + ".jpg")
    xml_path = os.path.join(xml_dir, stem + ".xml")

    img = np.array(Image.open(img_path).convert("RGB"))
    boxes, labels = parse_xml(xml_path)

    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.imshow(img)

    for box, lb in zip(boxes, labels):
        x1, y1, x2, y2 = box.tolist()
        w, h = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), w, h, fill=False, linewidth=2)
        ax.add_patch(rect)
        ax.text(x1, y1 - 5, classes[lb.item()], fontsize=10)

    ax.set_title(f"GT boxes check: {stem}")
    ax.axis("off")
    plt.show()

show_image_with_boxes(trainval_list[0])


✅ Dataset을 이렇게 구성하는 이유

torchvision detection 모델(SSD 포함)은 입력을 다음 형태로 받는다.

- images: 리스트 형태의 이미지 텐서들 (각각 [C,H,W])
- targets: 리스트 형태의 dict들
  - boxes: (N,4) XYXY
  - labels: (N,)
  - 추가로 image_id: 평가 시 추적을 위해 넣어주면 편하다

여기서 중요한 포인트는,
"이미지 증강을 할 때 박스도 같이 변환되어야 한다"는 것.
그래서 transforms.v2 + tv_tensors.BoundingBoxes를 사용한다.


▫️Detection Dataset + collate_fn

In [ ]:
class PetFaceDataset(Dataset):
    def __init__(self, image_list, transforms=None):
        self.image_list = image_list
        self.transforms = transforms

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        stem = self.image_list[idx]
        img_path = os.path.join(image_dir, stem + ".jpg")
        xml_path = os.path.join(xml_dir, stem + ".xml")

        image = Image.open(img_path).convert("RGB")
        boxes, labels = parse_xml(xml_path)

        # tv_tensors로 바꿔야 transforms.v2가 bbox까지 같이 변환해준다.
        image = tv_tensors.Image(image)
        bbox  = tv_tensors.BoundingBoxes(
            boxes,
            format=tv_tensors.BoundingBoxFormat.XYXY,
            canvas_size=image.shape[-2:]  # (H,W)
        )

        if self.transforms:
            image, bbox = self.transforms(image, bbox)
            boxes = bbox.data

        target = {
            "image_id": torch.tensor(idx),  # 평가 때 어떤 이미지인지 추적용(편의)
            "boxes": boxes,
            "labels": labels
        }
        return image, target

def collate_fn(batch):
    # detection은 이미지마다 객체 수가 달라서 텐서로 stack이 안 된다. 그래서 "리스트"로 묶어서 반환하는 collate_fn이 필요하다.
    images, targets = tuple(zip(*batch))
    return list(images), list(targets)


▫️Transform 정의(Train vs Val/Test)

In [ ]:
# Train: 다양한 환경(밝기, 구도)을 흉내내서 일반화를 조금이라도 올리기
# Val/Test: 랜덤 증강 없이, "평가 가능한 고정 전처리"만 적용

train_transform = v2.Compose([
    v2.RandomResizedCrop((300, 300), scale=(0.7, 1.0)),
    v2.RandomPerspective(distortion_scale=0.3, p=0.5),
    v2.RandomHorizontalFlip(p=0.5),

    v2.ToImage(),
    v2.ToDtype(dtype=torch.float32, scale=True),
    v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

test_transform = v2.Compose([
    v2.Resize((300, 300)),
    v2.ToImage(),
    v2.ToDtype(dtype=torch.float32, scale=True),
    v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])


▫️Train/Val 분할 + 클래스 불균형 보정(WeightedRandomSampler)

In [ ]:
# 간단하게 trainval을 train/val로 나눔
full_dataset = PetFaceDataset(trainval_list, transforms=train_transform)

val_ratio = 0.2
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


# Val은 증강을 빼고 고정 전처리로 다시 감싸는 게 안전
# val에 랜덤 증강이 들어가면 "측정값"이 흔들려서 비교가 어려워진다.
val_dataset.dataset.transforms = test_transform


# 클래스 불균형 보정
# 특정 클래스가 더 많으면, 모델이 그쪽만 맞추는 방향으로 학습이 기울 수 있다.
def get_sample_weights(dataset_subset):
    class_counts = {1: 0, 2: 0}  # cat, dog

    # subset은 random_split의 Subset 객체라서 dataset_subset[i]로 접근하면 (image, target)이 나온다.
    for i in range(len(dataset_subset)):
        _, target = dataset_subset[i]
        for lb in target["labels"]:
            if lb.item() in class_counts:
                class_counts[lb.item()] += 1

    # 역빈도 기반 weight
    total = sum(class_counts.values())
    class_weights = {k: (total / v) for k, v in class_counts.items() if v > 0}

    sample_weights = []
    for i in range(len(dataset_subset)):
        _, target = dataset_subset[i]
        main_class = target["labels"][0].item()  # 이 데이터는 보통 1개 객체(얼굴)라 0번째로 충분
        sample_weights.append(class_weights.get(main_class, 1.0))

    return torch.DoubleTensor(sample_weights), class_counts


sample_weights, class_counts = get_sample_weights(train_dataset)
print("class_counts (train):", class_counts)


sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)


▫️DataLoader 생성

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler, collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, collate_fn=collate_fn)

# test dataset은 test_list 기반으로 구성
test_dataset = PetFaceDataset(test_list, transforms=test_transform)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

print("loaders ready")


✅ SSD 모델에서 “분류 헤드 교체”가 왜 필요한가?

사전학습 SSD300(VGG16)은 COCO 같은 데이터셋의 클래스 수에 맞춰져 있다.
하지만 우리는 "background / cat / dog" 3클래스만 필요하다.

따라서 해야 할 핵심 작업은 딱 하나다.
- 분류 헤드(classification head)만 3클래스 출력으로 교체한다.

이걸 안 하면,
모델은 엉뚱한 클래스 공간에서 학습하게 되어 loss가 의미가 없어지거나 학습이 잘 안 된다.


▫️SSD300(VGG16) 로드 + classification head 교체

In [ ]:
num_classes = 3  # background, cat, dog

model = ssd300_vgg16(weights=SSD300_VGG16_Weights.DEFAULT)

# SSD300의 feature map별 채널 수는 관례적으로 아래처럼 사용
in_channels = [512, 1024, 512, 256, 256, 256]
num_anchors = model.anchor_generator.num_anchors_per_location()

# 분류 헤드 교체
model.head.classification_head = SSDClassificationHead(in_channels, num_anchors, num_classes)

model.to(device)
print("SSD300_VGG16 ready for fine-tuning")


✅ mAP를 왜 직접 구현했나?
- baseline에서 AP를 scikit-learn average_precision_score로 단순히 계산하면
"탐지 문제의 정석 평가"와 다르게 동작할 수 있어 찜찜한 부분이 생긴다.

탐지의 AP/mAP는 보통 다음 절차를 따른다.

1) 예측 박스들을 score 높은 순으로 정렬  
2) 각 예측이 GT 박스와 IoU 기준으로 매칭되는지 판단  
3) TP/FP 누적 → Precision/Recall 커브 생성  
4) 면적(AP) 계산 → 클래스별 AP 평균이 mAP

- 즉, 탐지 평가는 "박스 매칭 로직"이 핵심이라, 이를 직접 구현해두면 결과를 훨씬 더 납득하기 쉽다.


▫️IoU + AP + mAP@0.5 평가 함수

In [ ]:
def compute_iou_numpy(box1, box2):
    """
    IoU(Intersection over Union) 예시로 이해하기:
    - box1과 box2가 얼마나 '겹치는지'를 0~1로 수치화한 값
    - 완전 겹치면 1, 전혀 안 겹치면 0

    box: [x1,y1,x2,y2] (XYXY)
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_w = max(0.0, x2 - x1)
    inter_h = max(0.0, y2 - y1)
    inter_area = inter_w * inter_h

    area1 = max(0.0, (box1[2]-box1[0])) * max(0.0, (box1[3]-box1[1]))
    area2 = max(0.0, (box2[2]-box2[0])) * max(0.0, (box2[3]-box2[1]))

    union = area1 + area2 - inter_area + 1e-9
    return inter_area / union

def voc_ap(rec, prec):
    """
    VOC 방식 AP 계산:
    - Precision-Recall 곡선 아래 면적을 계산
    - 구현 포인트: precision을 오른쪽에서 왼쪽으로 monotonic하게 보정
    """
    mrec  = np.concatenate(([0.0], rec,  [1.0]))
    mpre  = np.concatenate(([0.0], prec, [0.0]))

    for i in range(len(mpre)-1, 0, -1):
        mpre[i-1] = max(mpre[i-1], mpre[i])

    idx = np.where(mrec[1:] != mrec[:-1])[0]
    ap = np.sum((mrec[idx+1] - mrec[idx]) * mpre[idx+1])
    return ap

def evaluate_map(preds, gts, iou_threshold=0.5, num_classes=3):
    """
    preds: list of dict
      - image_id, bbox(XYXY), score, label(1 or 2)
    gts: list of dict
      - image_id, boxes(N,4), labels(N,)
    """
    # GT를 image_id 기준으로 빠르게 찾기 위한 dict
    gt_by_image = {}
    for gt in gts:
        gt_by_image[gt["image_id"]] = {
            "boxes": gt["boxes"],
            "labels": gt["labels"],
            "detected": {1: set(), 2: set()}  # 클래스별로 어떤 GT가 매칭됐는지 기록
        }

    aps = []
    for cls in range(1, num_classes):  # 1~(num_classes-1): cat,dog
        cls_preds = [p for p in preds if p["label"] == cls]
        cls_preds = sorted(cls_preds, key=lambda x: x["score"], reverse=True)

        tp = np.zeros(len(cls_preds))
        fp = np.zeros(len(cls_preds))

        # 이 클래스의 GT 개수
        n_gt = 0
        for gt in gts:
            n_gt += np.sum(gt["labels"] == cls)

        for i, p in enumerate(cls_preds):
            image_id = p["image_id"]
            pred_box = p["bbox"]

            if image_id not in gt_by_image:
                fp[i] = 1
                continue

            gt_info = gt_by_image[image_id]
            gt_boxes = gt_info["boxes"]
            gt_labels = gt_info["labels"]

            # 이 이미지에서 이 클래스 GT만 뽑기
            cls_gt_idx = np.where(gt_labels == cls)[0]
            if len(cls_gt_idx) == 0:
                fp[i] = 1
                continue

            # IoU 최대 매칭 찾기
            best_iou = 0.0
            best_j = -1
            for j in cls_gt_idx:
                iou = compute_iou_numpy(pred_box, gt_boxes[j])
                if iou > best_iou:
                    best_iou = iou
                    best_j = int(j)

            # IoU 기준 충족 + 아직 매칭 안 된 GT면 TP
            if best_iou >= iou_threshold and best_j not in gt_info["detected"][cls]:
                tp[i] = 1
                gt_info["detected"][cls].add(best_j)
            else:
                fp[i] = 1

        # 누적합으로 PR 계산
        tp_cum = np.cumsum(tp)
        fp_cum = np.cumsum(fp)

        rec = tp_cum / (n_gt + 1e-9)
        prec = tp_cum / (tp_cum + fp_cum + 1e-9)

        ap = voc_ap(rec, prec)
        aps.append(ap)

    mAP = float(np.mean(aps)) if len(aps) > 0 else 0.0
    return mAP, aps




🟩 학습/검증 루프

In [ ]:
# Full Fine-tuning: backbone 포함 전체 파라미터를 업데이트
# 그래서 lr은 너무 크게 잡으면 쉽게 발산/불안정해질 수 있다.
# AdamW: weight decay로 과적합 완화에 도움을 기대

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

num_epochs = 50
best_map = 0.0

train_losses, val_losses, val_maps = [], [], []

# val GT를 미리 뽑아두면, 매 epoch 평가가 단순해짐
ground_truths_val = []
for _, target in val_dataset:
    ground_truths_val.append({
        "image_id": int(target["image_id"].item()),
        "boxes": target["boxes"].numpy(),
        "labels": target["labels"].numpy()
    })

for epoch in range(num_epochs):

    # Train
    model.train()
    running_loss = 0.0

    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        # SSD는 여러 loss(분류 + 위치)를 dict로 준다 → 합쳐서 최종 loss로 사용
        loss = sum(loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / max(1, len(train_loader))
    train_losses.append(avg_train_loss)


    # Val (loss + mAP)
    model.eval()
    val_loss = 0.0
    predictions_val = []

    with torch.no_grad():
        for images, targets in val_loader:
            images = [img.to(device) for img in images]
            targets_gpu = [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]

            # val loss도 보고 싶으면(네 분석처럼) train과 동일하게 loss 계산
            loss_dict = model(images, targets_gpu)
            loss = sum(loss_dict.values())
            val_loss += loss.item()

            # 예측은 target 없이 forward
            outputs = model(images)

            # outputs: 각 이미지마다 dict(boxes, labels, scores)
            for i, output in enumerate(outputs):
                img_id = int(targets[i]["image_id"].item())
                for j in range(len(output["boxes"])):
                    predictions_val.append({
                        "image_id": img_id,
                        "bbox": output["boxes"][j].cpu().numpy(),
                        "score": float(output["scores"][j].cpu().numpy()),
                        "label": int(output["labels"][j].cpu().numpy()),
                    })

    avg_val_loss = val_loss / max(1, len(val_loader))
    val_losses.append(avg_val_loss)

    val_map, aps = evaluate_map(predictions_val, ground_truths_val, iou_threshold=0.5, num_classes=NUM_CLASSES)
    val_maps.append(val_map)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val mAP: {val_map:.4f}")

    # Best 모델 저장
    if val_map > best_map:
        best_map = val_map
        torch.save(model.state_dict(), "best_ssd_model.pth")
        print(f"Best mAP Updated: {best_map:.4f}. Model Saved")

print("학습 완료")
print(f"최종 Best mAP: {best_map:.4f}")


'''
Epoch 1: Val mAP 0.1919

Epoch 5: Val mAP 0.9488

Epoch 10: Val mAP 0.9784

Epoch 34: Best mAP 0.9939

Epoch 50: Val mAP 0.9930

최종 출력: 최종 Best mAP: 0.9939
'''

▫️학습 곡선 시각화 (loss / mAP)

In [ ]:
# "mAP는 좋은데 loss가 생각보다 안 내려간다" 같은 상황이 있다.
# 탐지는 loss가 여러 개(분류+위치)라서, loss 자체보다 mAP가 더 목적에 가깝다.
# 그래도 곡선을 보면 학습이 정상적으로 진행되는지 감을 잡을 수 있다.

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(train_losses, label="train loss")
plt.plot(val_losses, label="val loss")
plt.title("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(val_maps, label="val mAP@0.5")
plt.title("mAP")
plt.legend()

plt.tight_layout()
plt.show()

print("best mAP:", best_map)


▫️테스트셋 예측 시각화(정성 평가)

In [ ]:
# 베스트 모델 로드
model.load_state_dict(torch.load("best_ssd_model.pth", map_location=device))
model.eval()

def denormalize(img_tensor):
    # v2.Normalize를 되돌려서 사람이 보기 좋게 만드는 함수
    mean = torch.tensor([0.485, 0.456, 0.406], device=img_tensor.device).view(3,1,1)
    std  = torch.tensor([0.229, 0.224, 0.225], device=img_tensor.device).view(3,1,1)
    x = img_tensor * std + mean
    x = torch.clamp(x, 0, 1)
    return x

@torch.no_grad()
def show_test_predictions(n_images=8, score_thr=0.5):
    shown = 0
    for images, targets in test_loader:
        images_gpu = [img.to(device) for img in images]
        outputs = model(images_gpu)

        for img_t, out in zip(images, outputs):
            img_np = denormalize(img_t.to(device)).permute(1,2,0).cpu().numpy()

            fig, ax = plt.subplots(1,1, figsize=(6,6))
            ax.imshow(img_np)
            ax.axis("off")

            for box, score, label in zip(out["boxes"], out["scores"], out["labels"]):
                if float(score) < score_thr:
                    continue
                x1,y1,x2,y2 = box.cpu().numpy()
                rect = patches.Rectangle((x1,y1), x2-x1, y2-y1, fill=False, linewidth=2)
                ax.add_patch(rect)
                ax.text(x1, y1-5, f"{classes[int(label)]} {float(score):.2f}", fontsize=10)

            plt.show()
            shown += 1
            if shown >= n_images:
                return

show_test_predictions(n_images=8, score_thr=0.5)


# 🟨 최종 결론 및 분석

## 1. 실험 요약
본 실험은 Oxford-IIIT Pet 데이터의 XML 바운딩박스를 이용해,
SSD300(VGG16) 기반으로 개/고양이 얼굴 영역을 검출하는 탐지 모델을 구축했다.

파이프라인은 다음 순서로 구성했다.
- 데이터 정합성 확인(이미지-XML 매칭)
- XML 파싱 검증(시각화 sanity check)
- detection dataset/dataloader 구성(박스 변환 포함)
- 클래스 불균형 보정(WeightedRandomSampler)
- SSD300 사전학습 모델 로드 후 분류 헤드 교체(3클래스)
- Full Fine-tuning 학습 + mAP@0.5 평가 + 테스트 정성 확인

---

## 2. 정량 결과
학습이 진행되면서 mAP가 빠르게 상승했고, 이후 0.99 수준에서 안정적으로 유지되는 것을 확인했다.

- Epoch 1: Val mAP 0.1919
- Epoch 5: Val mAP 0.9488
- Epoch 10: Val mAP 0.9784
- Epoch 34: Best Val mAP 0.9939
- Epoch 50: Val mAP 0.9930
- 최종 Best mAP: 0.9939

또한 train loss는 대략 6.93 → 1.34 수준으로 감소했고,
val loss는 1.92대(최저)까지 내려가며 전체적으로 학습이 정상적으로 수렴했다.

---

## 3. 관찰 및 해석
1) mAP는 매우 높게 나왔지만, loss와 완전히 같은 방향으로 움직이지는 않았다  
- 탐지 모델의 loss는 (분류 손실 + 위치 손실) 등 여러 요소가 합쳐진 값이라
  “loss가 조금 높은데도 mAP가 좋아지는” 상황이 충분히 발생할 수 있다.
- 따라서 이 문제의 목적(얼굴 검출 성능) 관점에서는 mAP를 1순위로 두는 판단이 타당하다.

2) 데이터 정합성 검사와 초기 시각화가 실험 품질에 결정적이었다  
- 이미지-XML 불일치가 섞이면 학습 자체가 깨질 수 있으므로,
  초반에 정합성 검사를 수행한 접근은 실전에서도 매우 중요하다.

3) 클래스 불균형은 샘플링 단계에서 먼저 완화하는 편이 안전했다  
- 데이터가 한 클래스에 치우치면 모델이 쉬운 방향으로 학습이 기울 수 있다.
- WeightedRandomSampler는 모델 구조를 바꾸지 않고도 학습 분포를 보정할 수 있어 적용 가치가 있다.

---

## 4. 한계 및 개선 방향
- mAP가 높은 이유가 “문제 난이도(얼굴이 비교적 크고 명확)”의 영향일 수 있으므로,
  더 어려운 조건(작은 얼굴, 가림, 다양한 조명)에서의 일반화 테스트가 추가로 필요하다.
- 향후 개선 방향
  - score threshold 조정 + NMS 설정 점검으로 FP를 더 줄이는 방향
  - 더 강한 증강(ColorJitter, Blur 등)의 영향 비교
  - IoU threshold(0.5 → 0.75 등)에서 성능이 유지되는지 추가 점검

---

## 5. 결론
SSD300 기반 개/고양이 얼굴 탐지 파이프라인을 end-to-end로 구축했으며,
mAP@0.5 기준 최고 0.9939의 성능을 확인했다.
또한 평가 로직(AP/mAP)을 직접 구성하여 결과에 대한 납득 가능성과 재현성을 확보했다.
